# Feature Engineering test ground
![Reverse Engineer](https://crystal-cdn3.crystalcommerce.com/photos/6337507/en_UJy7SBBy4F.png)

In [39]:
import numpy as np
import pandas as pd
import math
from model.master_transmuter import *
from unit_tests import *
from query import *

In [50]:
cards_df = pd.read_csv('data/all_vintage_cards.csv')
creature = cards_df[cards_df['name']=='Arcanis the Omnipotent'].iloc[0]
spell = cards_df[cards_df['name']=='Momentary Blink'].iloc[0]
weird = cards_df[cards_df['name']=='Tarmogoyf'].iloc[0]
tests = pd.concat([creature, spell, weird], axis=1).T
tests

FileNotFoundError: File b'scrape/all_vintage_cards.csv' does not exist

In [ ]:
pt_mask = np.array([False, False, True])
tests['variable_pt'] = True
tests['variable_pt'][~pt_mask] = False
tests

In [ ]:
pt_mask = np.array([True, False, False])
tests['p:t'] = tests[pt_mask]['power'].astype('int')/tests[pt_mask]['toughness'].astype('int')
tests

In [ ]:
def pt_type(row):
    if (type(row['power']) == type('str')) and (type(row['toughness']) == type('str')):
        if '*' in row['power']+row['toughness']:
            return 'variable'
        return 'static'
    return 'none'
def power_to_int(row):
    if row['pt_type']=='static': 
        return int(row['power'])
    else:
        return row['power']
def tough_to_int(row):
    if row['pt_type']=='static':
        return int(row['toughness'])
    else:
        return row['toughness']
df = tests.copy()
# Create pt_type feature, convert static pts to ints
df['pt_type'] = df.apply(pt_type, axis=1)
df['power'] = df.apply(power_to_int, axis=1)
df['toughness'] = df.apply(tough_to_int, axis=1)

mask = df['pt_type']=='static'

df['p:t'] = df[mask]['power']/df[mask]['toughness']
df['p+t'] = df[mask]['power']+df[mask]['toughness']
df['p*t'] = df[mask]['power']*df[mask]['toughness']
df['sqrt_pt'] = math.sqrt(df[mask]['p*t'])
df['avg_pt'] = (df[mask]['power']+df[mask]['toughness'])/2
df['cmc:p+t'] = df[mask]['cmc']/df[mask]['p+t']
df['cmc:p*t'] = df[mask]['cmc']/df[mask]['sqrt_pt']
df

In [ ]:
df['p*t'].apply(np.sqrt)

In [ ]:
math.sqrt((df[mask]['power']*df[mask]['toughness']))

In [ ]:
test_feature_engineering(cards_df)

In [116]:
test_card.apply(get_recent_price, axis=1)

('Polyraptor', 'Rivals of Ixalan', '1542070311053', 2.3)


2857    (1542070311053, 2.3)
dtype: object

In [122]:
test_card.apply(get_recent_price, axis=1)[:]

('Polyraptor', 'Rivals of Ixalan', '1542070311053', 2.3)


2857    (1542070311053, 2.3)
dtype: object

# Testing Pipelines

In [6]:
X = pd.read_csv('data/all_vintage_cards-rare_recent.csv', low_memory=False)

In [8]:
cards = [x.split('//') for x in X['type_line'].unique()]
cards[:10]

[['Legendary Creature — Minotaur Cleric'],
 ['Sorcery'],
 ['Legendary Creature — Human Cleric'],
 ['Land'],
 ['Legendary Artifact Creature — Construct'],
 ['Legendary Artifact'],
 ['Artifact Creature — Assembly-Worker'],
 ['Legendary Artifact — Equipment'],
 ['Artifact'],
 ['Artifact — Equipment']]

In [9]:
cards[0]

['Legendary Creature — Minotaur Cleric']

In [23]:
subcard = cards[0]
types = subcard[0].split(' — ')
types

['Legendary Creature', 'Minotaur Cleric']

In [13]:
card_types = set(['Creature','Land','Instant','Sorcery','Enchantment','Artifact','Planeswalker'])
sub_types = set()
type_mods = set()

for card in cards:
    for subcard in card:
        types = subcard.split(' — ')
        type_mods.update(set(types[0].split()) - card_types)
        try:
            sub_types.update(set(types[1].split()))
        except:
            pass
print(type_mods)
print(sub_types)

{'Snow', 'Legendary', 'Tribal', 'World'}
{'Vehicle', 'Zombie', 'Berserker', 'Sliver', 'Phoenix', 'Atog', 'Ninja', 'Construct', 'Cleric', 'Troll', 'Bird', 'Rigger', 'Ox', 'Faerie', 'Orc', 'Treefolk', 'Archer', 'Advisor', 'Whale', 'Horror', 'Scarecrow', 'Shapeshifter', 'Fox', 'Masticore', 'Gorgon', 'Insect', 'Hippogriff', 'Griffin', 'Mutant', 'Spellshaper', 'Snake', 'Thalakos', 'Wurm', 'Dinosaur', 'Horse', 'Barbarian', 'Liliana', 'Metathran', 'Knight', 'Thrull', 'Golem', 'Spider', 'Lizard', 'Naga', 'Turtle', 'Devil', 'Jace', 'Orgg', 'Trap', 'Spike', 'Skeleton', 'Ouphe', 'Mongoose', 'Kirin', 'Djinn', 'Thopter', 'Fungus', 'Juggernaut', 'Eldrazi', 'Rabbit', 'Nephilim', 'Dryad', 'Jellyfish', 'Human', 'Egg', 'Squirrel', 'Pirate', 'Warrior', 'Minion', 'Sphinx', 'Gnome', 'Giant', 'Island', 'Ajani', 'Viashino', 'Arcane', 'Dreadnought', 'Carrier', 'Specter', 'Hound', 'Rogue', 'Centaur', 'Pegasus', 'Efreet', 'Wolf', 'Kithkin', 'Elemental', 'Boar', 'Serpent', 'Illusion', 'Bringer', 'Druid', 'Brushw

In [31]:
card_types or set(types[0].split())

{'Artifact',
 'Creature',
 'Enchantment',
 'Instant',
 'Land',
 'Planeswalker',
 'Sorcery'}

In [34]:
(set(types[0].split()) - card_types)

{'Legendary'}

In [21]:
def type_sets(row):
    card = row['type_line'].split('//')
    for subcard in card:
        print(subcard)
    return row
df = X.apply(type_sets, axis=1)

Legendary Creature — Minotaur Cleric
Legendary Creature — Minotaur Cleric
Sorcery
Legendary Creature — Human Cleric
Land
Land
Land
Land
Land
Land
Legendary Artifact Creature — Construct
Legendary Artifact
Artifact Creature — Assembly-Worker
Legendary Artifact — Equipment
Artifact
Artifact — Equipment
Legendary Artifact — Equipment
Legendary Sorcery
Legendary Enchantment
Legendary Creature — Human Wizard
Legendary Creature — Elf Warrior
Legendary Creature — Human Knight
Creature — Elemental
Creature — Dinosaur
Sorcery
Creature — Elf Knight
Enchantment — Saga
Legendary Creature — Elf Druid
Legendary Sorcery
Creature — Giant Warrior
Legendary Creature — Goblin
Creature — Goblin
Legendary Sorcery
Enchantment
Creature — Goblin Warrior
Enchantment — Saga
Legendary Sorcery
Legendary Creature — Avatar
Enchantment — Saga
Legendary Enchantment
Legendary Creature — Vampire
Legendary Creature — Zombie Knight
Creature — Shade
Legendary Creature — Djinn
Creature — Djinn
Enchantment
Legendary Creatur

In [35]:
typeline = TypelineTransformer()
typeline_test = typeline.transform(X)
typeline_test.head()

,Unnamed: 0,id,cardname,setname,type_line,mana_cost,rarity,oracle_text,power,toughness,...,card_types,mod_types,sub_types,Instant,Artifact,Land,Sorcery,Enchantment,Creature,Planeswalker
0,0,3923708a-60b8-4fb0-beb2-9ab18f5fd381,Firesong and Sunspeaker,Dominaria,Legendary Creature — Minotaur Cleric,{4}{R}{W},rare,Red instant and sorcery spells you control hav...,4,6,...,{Creature},{Legendary},"{Minotaur, Cleric}",0,0,0,0,0,1,0
1,1,f1e849c3-f357-4e81-a580-be5056bed51b,Chandra's Outburst,Dominaria,Sorcery,{3}{R}{R},rare,Chandra's Outburst deals 4 damage to target pl...,NaN,NaN,...,{Sorcery},{},{},0,0,0,1,0,0,0
2,2,c638677c-2b92-4d0c-b61c-598b5a843844,"Niambi, Faithful Healer",Dominaria,Legendary Creature — Human Cleric,{1}{W}{U},rare,"When Niambi, Faithful Healer enters the battle...",2,2,...,{Creature},{Legendary},"{Human, Cleric}",0,0,0,0,0,1,0
3,3,ba05cf47-9823-41f9-b893-321ea89e473e,Woodland Cemetery,Dominaria,Land,NaN,rare,Woodland Cemetery enters the battlefield tappe...,NaN,NaN,...,{Land},{},{},0,0,1,0,0,0,0
4,4,6bc412fc-e9d4-4283-bd4a-811384e79b5c,Sulfur Falls,Dominaria,Land,NaN,rare,Sulfur Falls enters the battlefield tapped unl...,NaN,NaN,...,{Land},{},{},0,0,1,0,0,0,0


In [17]:
X['type_line'][0].split('//')

['Legendary Creature — Minotaur Cleric']

In [37]:
X['color_identity'].unique()

array(["['R', 'W']", "['R']", "['U', 'W']", "['B', 'G']", "['R', 'U']",
       "['B', 'W']", "['G', 'U']", "['B']", '[]',
       "['B', 'G', 'R', 'U', 'W']", "['G', 'R']", "['G']", "['B', 'R']",
       "['U']", "['W']", "['G', 'W']", "['B', 'U']", "['B', 'R', 'U']",
       "['B', 'U', 'W']", "['G', 'U', 'W']", "['B', 'G', 'R']",
       "['B', 'G', 'W']", "['B', 'R', 'W']", "['G', 'R', 'U']",
       "['B', 'G', 'U']", "['R', 'U', 'W']", "['G', 'R', 'W']",
       "['B', 'R', 'U', 'W']", "['B', 'G', 'U', 'W']",
       "['G', 'R', 'U', 'W']"], dtype=object)

In [38]:
X['colors'].unique()

array(["['R', 'W']", "['R']", "['U', 'W']", '[]', "['B', 'W']",
       "['R', 'U', 'W']", "['G', 'R']", "['G']", "['B']", "['U']",
       "['W']", "['B', 'R']", "['G', 'W']", "['R', 'U']", "['B', 'G']",
       "['G', 'U']", "['B', 'U']", "['B', 'U', 'W']", "['G', 'U', 'W']",
       "['B', 'R', 'U']", "['B', 'G', 'R']", "['B', 'G', 'R', 'U', 'W']",
       "['B', 'R', 'W']", "['B', 'G', 'U']", "['G', 'R', 'U']",
       "['B', 'G', 'W']", "['G', 'R', 'W']", "['B', 'R', 'U', 'W']",
       "['B', 'G', 'U', 'W']", "['G', 'R', 'U', 'W']"], dtype=object)

In [40]:
colorid = ColorIDTransformer()
color_df = colorid.transform(X)
color_df.head()

,Unnamed: 0,id,cardname,setname,type_line,mana_cost,rarity,oracle_text,power,toughness,...,reprint,layout,legalities,timestamp,price,G,W,R,U,B
0,0,3923708a-60b8-4fb0-beb2-9ab18f5fd381,Firesong and Sunspeaker,Dominaria,Legendary Creature — Minotaur Cleric,{4}{R}{W},rare,Red instant and sorcery spells you control hav...,4,6,...,False,normal,"{'standard': 'legal', 'future': 'legal', 'fron...",1528476980380,11.5,0,1,1,0,0
1,1,f1e849c3-f357-4e81-a580-be5056bed51b,Chandra's Outburst,Dominaria,Sorcery,{3}{R}{R},rare,Chandra's Outburst deals 4 damage to target pl...,NaN,NaN,...,False,normal,"{'standard': 'legal', 'future': 'legal', 'fron...",1542041780379,0.1,0,0,1,0,0
2,2,c638677c-2b92-4d0c-b61c-598b5a843844,"Niambi, Faithful Healer",Dominaria,Legendary Creature — Human Cleric,{1}{W}{U},rare,"When Niambi, Faithful Healer enters the battle...",2,2,...,False,normal,"{'standard': 'legal', 'future': 'legal', 'fron...",1542041780379,0.1,0,1,0,1,0
3,3,ba05cf47-9823-41f9-b893-321ea89e473e,Woodland Cemetery,Dominaria,Land,NaN,rare,Woodland Cemetery enters the battlefield tappe...,NaN,NaN,...,True,normal,"{'standard': 'legal', 'future': 'legal', 'fron...",1542041780378,6.0,1,0,0,0,1
4,4,6bc412fc-e9d4-4283-bd4a-811384e79b5c,Sulfur Falls,Dominaria,Land,NaN,rare,Sulfur Falls enters the battlefield tapped unl...,NaN,NaN,...,True,normal,"{'standard': 'legal', 'future': 'legal', 'fron...",1542041780378,11.0,0,0,1,1,0
